<a href="https://colab.research.google.com/github/khalil-alexander/get_forecast/blob/main/FuturePositionISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color="red"> Pip install Astropy in google colab <font/>

In [1]:
!pip install astropy

## <font color="red"> Install Moudles in google colab <font/>

In [2]:
from astropy import coordinates as coord
from astropy import units as u
from astropy import time
from astropy.time import Time
from astropy.coordinates import Angle

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests as reqs
import xml.etree.ElementTree as ET
import io
import pprint
import json
import datetime

## <font color="red"> Converting coordinates to Latitude and Longitude ISS</font>

In [4]:
now = Time("2024-03-07 00:46:00.000", scale='utc')
xyz = [1155.746046202530, -6632.420367726780, 953.533229633281]
cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)



gcrs = coord.GCRS(cartrep, obstime = now)
itrs = gcrs.transform_to(coord.ITRS(obstime = now))
loc = coord.EarthLocation(*itrs.cartesian.xyz)

print(loc.lat, loc.lon, loc.height)

8d07m57.90103915s 103d26m20.85144985s 421.8445702007271 km


## <font color="red"> Creating functions to clean up Latitude and Longitude ISS</font>

In [5]:
col_name = ['Date','x units(km)','y units(km)','z units(km)','x_dot(km/s)','y_dot(km/s)','z_dot(km/s)']
# Challenge to have the columns read under beautiful soup
xml_df = pd.DataFrame(columns=col_name)
xml_df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)


In [6]:
def create_latlon_positions(date: str, xyz: list[float]):
    now = Time(date, scale='utc')
    cartrep = coord.CartesianRepresentation(*xyz, unit=u.km)

    gcrs = coord.GCRS(cartrep, obstime = now)
    itrs = gcrs.transform_to(coord.ITRS(obstime = now))
    loc = coord.EarthLocation(*itrs.cartesian.xyz)

    return Angle(loc.lat).deg, Angle(loc.lon).deg


In [7]:
def get_latlon(row: pd.Series):
    date = row['date'].strftime('%Y-%m-%d %H:%M:%S')
    xyz = [row['x'], row['y'], row['z']]
    return create_latlon_positions(date, xyz)




In [8]:
def read_xml_DataFrame(url):
    """
    Do web scraping from an XML file to obtain time the International Space
    Station will be at a specfic postion and the speed it is going at.

    Parameters
    ----------
    url : String
        The url of the data in XML format you want to extract from.

    Return
    ------
    df : Pandas DataFrame
        The DataFrame containing the time the International Space Station is at
        a specfic position and the speed it is traveling.
    """
    # Get the xml request
    xml = reqs.get(url)

    # Use BeautifulSoup to Organize the XML file
    soup = BeautifulSoup(xml.content,"lxml")

    # Finds all the instances 'statevector' appeared in the xml file
    state_tag = soup.find_all('statevector')

    col_name = ['Date','x units(km)',
                'y units(km)','z units(km)',
                'x_dot(km/s)',
                'y_dot(km/s)','z_dot(km/s)']
    df = pd.DataFrame(columns=col_name)

    #Loops thorugh all elements to add rows in the empty xml_df
    for element in state_tag:
        # Turns element into text and splits the '\n'
        element = element.text.split('\n')
        # Deletes First empty String
        element.pop(0)
        # Deletes Last empty Sting
        element.pop(-1)
        # Removes the 'Z' in the elements first index
        element[0] = element[0][:-1]

        # Appending list into DataFrame by adding 1 row to the DataFrame
        #df = xml_df.append(pd.DataFrame(element, columns=col_name), ignore_index=True)
        df.loc[len(df)] = element


    # Turns the "Date" collumn into a Datetime object
    df['Date'] = pd.to_datetime(xml_df['Date'], format='%Y-%jT%H:%M:%S.%f')
    # Sets the "Date" collumn into a Datetime object
    #df.set_index("Date", inplace=True)

    # Returns the Dataframe
    return df

In [9]:
xml_url = "https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.xml"
%time df = read_xml_DataFrame(xml_url)

<ipython-input-8-7ae32a981933>:21: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(xml.content,"lxml")


CPU times: user 8.69 s, sys: 73.2 ms, total: 8.77 s
Wall time: 9.72 s


In [10]:
df

,Date,x units(km),y units(km),z units(km),x_dot(km/s),y_dot(km/s),z_dot(km/s)
0,NaT,-4207.6218330000002,500.407692,-5312.3958579999999,-1.2087917016530001,-7.5559985453309997,0.243217872256
1,NaT,-4341.6662696539597,-1309.20505135849,-5061.5767003754099,0.098591334701290001,-7.4323103937428501,1.83422268696472
2,NaT,-4160.8287176081403,-3023.8599672820701,-4442.6192989889696,1.3993440813959399,-6.76939995240022,3.2924029264916599
3,NaT,-3678.0488835309202,-4519.06622967363,-3500.35738989838,2.5994765962735098,-5.6145530783570496,4.5119665300729901
4,NaT,-2928.1416836870799,-5686.0469317759898,-2303.1552746478401,3.6117783935835699,-4.0507468890015899,5.4038155308844704
...,...,...,...,...,...,...,...
5399,NaT,458.73090781335901,-4193.1938477624399,5322.4338169791999,7.6236976278148703,0.73809754966782004,-0.071188316936819998
5400,NaT,2249.5608803007199,-3865.9134535613898,5111.7125715646798,7.2088929014410503,1.9725957860557,-1.6740652575638699
5401,NaT,3876.9967165425401,-3257.82841136139,4528.6463006095501,6.2703186753381104,3.0639268791760998,-3.1551772104684002
5402,NaT,5222.7391519114299,-2413.0487647598902,3615.61877289528,4.8756658980804701,3.93303357428332,-4.4069472876329199


In [ ]:
# Date needs to be fixed